In [1]:
%pip install pyyaml

Note: you may need to restart the kernel to use updated packages.


In [7]:
import json
from pathlib import Path

import yaml

from src.axiomatic_kernel import (
    AxiomKernel,
    VariableSchema,
    AxiomDefinition,
    DecisionLogger,
)
from src.nl_rule_parser import (
    build_axiom_from_nl,
    RuleParseError,
)
from src.explanation_engine import (
    DecisionExplainer,
    ExplanationConfig,
)
from src.rules_io import load_ruleset_from_file, apply_ruleset_to_kernel

In [8]:


# === 1) SCHEMA DOPASOWANA DO FRAUD_RULES (tylko typy obsługiwane przez kernel) ===
schema = [
    VariableSchema("amount", "int", "Kwota transakcji w jednostkach minimalnych."),
    VariableSchema("tx_count_24h", "int", "Liczba transakcji w ostatnich 24h."),
    VariableSchema("is_pep", "bool", "Czy klient jest PEP."),
    VariableSchema("is_suspicious", "bool", "Czy transakcja jest podejrzana."),
]

logger = DecisionLogger("logs/fraud_rules_demo.jsonl")

kernel = AxiomKernel(
    schema=schema,
    decision_variable="is_suspicious",
    logger=logger,
    rule_version="fraud_rules_v1",
)

# === 2) Wczytanie rulesetu z pliku YAML ===
rules_path = Path("rules") / "fraud_rules_v1.yaml"
rules_path.parent.mkdir(exist_ok=True)

ruleset = load_ruleset_from_file(rules_path)

print(f"📘 Wczytano ruleset: {ruleset.ruleset_id} (v{ruleset.version})")
print(f"Opis: {ruleset.description}")
print(f"Liczba reguł w pliku (łącznie): {len(ruleset.rules)}")

# === 3) Nałożenie rulesetu na kernel ===
summary = apply_ruleset_to_kernel(
    kernel=kernel,
    ruleset=ruleset,
    schema=schema,                         # 👈 NOWE: przekazujemy schema
    decision_field_fallback="is_suspicious",  # 👈 NOWE: ta sama zmienna co decision_variable
    strict=True,                           # przerwij przy pierwszym błędzie
    extra_metadata={"domain": "fraud-demo"},
)

print("\n📊 Podsumowanie ładowania reguł:")
print(f"- total_rules:   {summary.total_rules}")
print(f"- enabled_rules: {summary.enabled_rules}")
print(f"- loaded_rules:  {summary.loaded_rules}")
print(f"- skipped_rules: {summary.skipped_rules}")
print(f"- errors:        {summary.errors}")


📘 Wczytano ruleset: fraud_rules_v1 (v1.0.0)
Opis: Podstawowe reguły fraud / AML
Liczba reguł w pliku (łącznie): 3

📊 Podsumowanie ładowania reguł:
- total_rules:   3
- enabled_rules: 2
- loaded_rules:  2
- skipped_rules: 1
- errors:        {}


In [ ]:
# Katalog i plik z regułami dla tego notebooka
rules_dir = Path("rules")
rules_dir.mkdir(exist_ok=True)

rules_file = rules_dir / "notebook_rules.yaml"

# Domyślny zestaw reguł – użyty tylko, jeśli plik nie istnieje.
default_rules_yaml = """rules:
  - id: "nl_high_risk_flag"
    text: "If amount > 10000 and risk_score > 5 then flag = true"
  - id: "nl_low_risk_clear"
    text: "If risk_score <= 2 then flag = false"
"""

if not rules_file.exists():
    rules_file.write_text(default_rules_yaml, encoding="utf-8")
    print(f"Utworzono domyślny plik z regułami: {rules_file}")
else:
    print(f"Używam istniejącego pliku z regułami: {rules_file}")

# Wczytanie reguł z pliku YAML
with rules_file.open("r", encoding="utf-8") as f:
    rules_data = yaml.safe_load(f)

rules_list = rules_data.get("rules", [])

axioms = []
for raw_rule in rules_list:
    rule_id = raw_rule["id"]
    text = raw_rule["text"]

    axiom = build_axiom_from_nl(
        rule_id=rule_id,
        text=text,
        schema=schema,
        decision_field_fallback="flag",
    )
    kernel.add_axiom_safe(axiom)
    axioms.append(axiom)

print("Dodane reguły:", [a.id for a in axioms])


In [9]:
explainer = DecisionExplainer(ExplanationConfig(language="pl"))

# Przypadek wysokiego ryzyka – powinno być FLAGGED
case_flagged = {"amount": 15_000, "risk_score": 7}
bundle_flagged = kernel.evaluate(case_flagged)

print("=== RAW BUNDLE (FLAGGED) ===")
print(json.dumps(bundle_flagged, indent=2, ensure_ascii=False))

print("\n=== WYJAŚNIENIE (FLAGGED) ===")
print(explainer.explain(bundle_flagged).to_text(language="pl"))



# Przypadek niskiego ryzyka – powinno być CLEAN
case_clean = {"amount": 500, "risk_score": 1}
bundle_clean = kernel.evaluate(case_clean)

print("=== RAW BUNDLE (CLEAN) ===")
print(json.dumps(bundle_clean, indent=2, ensure_ascii=False))

print("\n=== WYJAŚNIENIE (CLEAN) ===")
print(explainer.explain(bundle_clean).to_text(language="pl"))


=== RAW BUNDLE (FLAGGED) ===
{
  "decision_status": "SAT",
  "decision": "FLAGGED",
  "facts": {
    "amount": 15000
  },
  "model": {
    "amount": 15000,
    "tx_count_24h": 0,
    "is_pep": false,
    "is_suspicious": true
  },
  "satisfied_axioms": [
    {
      "id": "fraud.high_amount",
      "description": "IF amount > 10000 THEN is_suspicious = TRUE",
      "holds": true,
      "antecedent_true": true
    },
    {
      "id": "fraud.velocity",
      "description": "IF tx_count_24h > 5 THEN is_suspicious = TRUE",
      "holds": true,
      "antecedent_true": false
    }
  ],
  "violated_axioms": [],
  "active_axioms": [
    {
      "id": "fraud.high_amount",
      "description": "IF amount > 10000 THEN is_suspicious = TRUE",
      "holds": true,
      "antecedent_true": true
    }
  ],
  "inactive_actions": [
    {
      "id": "fraud.velocity",
      "description": "IF tx_count_24h > 5 THEN is_suspicious = TRUE",
      "holds": true,
      "antecedent_true": false
    }
  ],
  "

In [10]:
from z3 import Implies  # type: ignore

unsat_kernel = AxiomKernel(
    schema=schema,
    decision_variable="flag",
    logger=None,
    rule_version="demo_unsat_v1",
)

def rule_flag_true(vars_z3):
    amount = vars_z3["amount"]
    flag = vars_z3["flag"]
    return Implies(amount > 10_000, flag == True)

def rule_flag_false(vars_z3):
    amount = vars_z3["amount"]
    flag = vars_z3["flag"]
    return Implies(amount > 10_000, flag == False)

unsat_kernel.add_axiom(
    AxiomDefinition(
        id="amount_flag_true",
        description="If amount > 10000 then flag must be True.",
        build_constraint=rule_flag_true,
    )
)
unsat_kernel.add_axiom(
    AxiomDefinition(
        id="amount_flag_false",
        description="If amount > 10000 then flag must be False.",
        build_constraint=rule_flag_false,
    )
)

case_conflict = {"amount": 15_000, "risk_score": 5}
bundle_unsat = unsat_kernel.evaluate(case_conflict)

print("=== RAW BUNDLE (UNSAT) ===")
print(json.dumps(bundle_unsat, indent=2, ensure_ascii=False))

print("\n=== WYJAŚNIENIE (UNSAT) ===")
print(explainer.explain(bundle_unsat).to_text(language="pl"))


Kernel evaluation error
Traceback (most recent call last):
  File "/home/jupyter/olga_zydziak/version_beta/Folder/casual_model/Axiomatic-learning/src/axiomatic_kernel.py", line 375, in evaluate
    constraint = axiom.build_constraint(self._variables)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/tmp/ipykernel_17759/431343928.py", line 12, in rule_flag_true
    flag = vars_z3["flag"]
           ~~~~~~~^^^^^^^^
KeyError: 'flag'


=== RAW BUNDLE (UNSAT) ===
{
  "decision_status": "ERROR",
  "decision": "ERROR",
  "facts": {
    "amount": 15000,
    "risk_score": 5
  },
  "model": {},
  "satisfied_axioms": [],
  "violated_axioms": [],
  "active_axioms": [],
  "inactive_actions": [],
  "conflicting_axioms": [],
  "rule_version": "demo_unsat_v1",
  "error": "'flag'"
}

=== WYJAŚNIENIE (UNSAT) ===
Wystąpił błąd podczas ewaluacji reguł. Kluczowe dane wejściowe: amount=15000, risk_score=5.

Błąd techniczny: 'flag'
